# Multiclass Prediction in Deep Learning Models

## Read in the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [3]:
from google.colab import files
uploaded = files.upload()

Saving meet_or_beat.csv to meet_or_beat.csv


In [4]:
df = pd.read_csv('meet_or_beat.csv')

In [5]:
df.shape

(71968, 5)

In [6]:
df.head(3)

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104


## Preparing the Data

In [7]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan
df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome'] = 'beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome'] = 'lose'
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat


In [8]:
# Preview the output variable 
y = df['earnings_outcome']
y

0        beat
1        lose
2        lose
3        lose
4        beat
         ... 
71963    beat
71964    beat
71965    lose
71966    beat
71967    meet
Name: earnings_outcome, Length: 71968, dtype: object

In [9]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [10]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [11]:
# Convert labeled integers to a Keras `categorical` data type
from keras.utils.np_utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)


In [12]:
# Specify X (predictor) variables
X = df[['forecasted_eps',
              'before_total_returns','noOfEsts']]
X.head(3)

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0


In [13]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [14]:
# Preview the encoded data we're trying to predict
pd.DataFrame(y_train).head(5)

,0,1,2
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


In [15]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float32

In [16]:
# Save the count of unique predictor variables for use in model
number_of_predictors = len(X.columns)
len(X.columns)

3

## Building, Fitting and Predicting with Multiple Classes

In [17]:
# Build the neural network layers
model = Sequential()
model.add(Dense(9, input_dim=number_of_predictors, activation='relu'))
model.add(Dense(6, activation='relu'))

In [18]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 
model.add(Dense(number_of_classes, activation='softmax'))

In [19]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="categorical_crossentropy", 
              optimizer= "adam", 
              metrics=['categorical_accuracy'])


In [20]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 60        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Fit the model
model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=1000,
                    shuffle=True)

Epoch 1/20
54/54 [==============================] - 1s 3ms/step - loss: 1.9290 - categorical_accuracy: 0.0876
Epoch 2/20
54/54 [==============================] - 0s 3ms/step - loss: 1.3662 - categorical_accuracy: 0.2372
Epoch 3/20
54/54 [==============================] - 0s 3ms/step - loss: 1.0905 - categorical_accuracy: 0.4644
Epoch 4/20
54/54 [==============================] - 0s 2ms/step - loss: 0.9282 - categorical_accuracy: 0.6031
Epoch 5/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8893 - categorical_accuracy: 0.6057
Epoch 6/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8799 - categorical_accuracy: 0.6072
Epoch 7/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8744 - categorical_accuracy: 0.6071
Epoch 8/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8699 - categorical_accuracy: 0.6073
Epoch 9/20
54/54 [==============================] - 0s 3ms/step - loss: 0.8669 - categorical_accuracy: 0.6076
Epoch 10/2

In [22]:
# Evaluate model on the test data
model.evaluate(X_test,y_test, verbose=2)

563/563 - 1s - loss: 0.8462 - categorical_accuracy: 0.6124 - 820ms/epoch - 1ms/step


[0.8461712002754211, 0.6124388575553894]

In [23]:
# Save predictions on the test data
predictions = model.predict(X_test)
predictions

array([[0.57655954, 0.3177183 , 0.10572211],
       [0.7998804 , 0.15021634, 0.04990332],
       [0.58443475, 0.33530888, 0.08025643],
       ...,
       [0.6461848 , 0.25360936, 0.10020576],
       [0.5704268 , 0.33706132, 0.09251193],
       [0.51569307, 0.39298046, 0.09132656]], dtype=float32)

In [24]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['beat', 'beat', 'beat', ..., 'beat', 'beat', 'beat'], dtype=object)

In [26]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

beat    17586
lose      406
dtype: int64